In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Min-entropy analysis with dynamic parameters - MULTIPLE n_history VERSION
Generate tables, CSV files, TeX output, and visualizations
"""

import math
import numpy as np
from typing import Tuple, List, Dict
import os
import sys
import datetime
import csv

# ---------- Standard normal distribution functions ----------
SQRT2 = math.sqrt(2.0)
SQRT2PI = math.sqrt(2.0 * math.pi)

def phi(x: float) -> float:
    """Standard normal PDF"""
    return math.exp(-0.5 * x * x) / SQRT2PI

def Phi(x: float) -> float:
    """Standard normal CDF"""
    return 0.5 * (1.0 + math.erf(x / SQRT2))

# ---------- Correlation model ----------
def rho_lags(n: int, alpha: float, beta: float) -> np.ndarray:
    """Return correlation function rho[1..n]"""
    k = np.arange(1, n + 1, dtype=float)
    return alpha * np.power(k, -beta)

# ---------- Min-entropy calculation ----------
def hmin_from_p(p: float) -> float:
    """Compute min-entropy from conditional probability p"""
    p = float(np.clip(p, 1e-12, 1.0 - 1e-12))
    q = max(p, 1.0 - p)
    return -math.log2(q)

# ---------- First-order approximation of p ----------
def p_cond_first_order_all_same(n: int, r: float, alpha: float, beta: float, b: int = -1) -> float:
    """
    First-order approximation of conditional probability p(b_n=b | b_0=...=b_{n-1}=b)
    """
    rho = rho_lags(n, alpha=alpha, beta=beta)
    s1 = float(np.sum(rho))
    
    if abs(r) < 1e-14:
        p = 0.5 + (1.0 / math.pi) * s1
    else:
        p0 = Phi(b * r)
        p0 = min(max(p0, 1e-15), 1.0 - 1e-15)
        p = p0 + (phi(r) ** 2 / p0) * s1
    
    return float(np.clip(p, 1e-12, 1.0 - 1e-12))

# ---------- Toeplitz correlation matrix ----------
def toeplitz_corr_matrix(n: int, alpha: float, beta: float) -> np.ndarray:
    """Build (n+1)x(n+1) Toeplitz correlation matrix"""
    dim = n + 1
    rho = rho_lags(n, alpha=alpha, beta=beta)
    first_row = np.concatenate(([1.0], rho))
    R = np.empty((dim, dim), dtype=float)
    for i in range(dim):
        for j in range(dim):
            R[i, j] = first_row[abs(i - j)]
    return R

def cholesky_with_jitter(R: np.ndarray, jitter0: float = 1e-12, max_tries: int = 12) -> np.ndarray:
    """Cholesky decomposition with jitter"""
    jitter = jitter0
    I = np.eye(R.shape[0], dtype=float)
    for _ in range(max_tries):
        try:
            return np.linalg.cholesky(R + jitter * I)
        except np.linalg.LinAlgError:
            jitter *= 10.0
    raise np.linalg.LinAlgError("Cholesky decomposition failed")

# ---------- Monte Carlo estimation of p ----------
def p_cond_monte_carlo_all_same(
    n: int,
    r: float,
    alpha: float,
    beta: float,
    b: int = -1,
    num_samples: int = 50000,
    seed: int = 0,
) -> float:
    """
    Monte Carlo estimation of conditional probability
    """
    rng = np.random.default_rng(seed)
    dim = n + 1
    
    R = toeplitz_corr_matrix(n, alpha=alpha, beta=beta)
    L = cholesky_with_jitter(R)
    
    eps = rng.standard_normal(size=(dim, num_samples))
    Z = (L @ eps).T
    V = Z + r
    
    B = np.where(V >= 0.0, 1, -1)
    past_ok = np.all(B[:, :n] == b, axis=1)
    denom = int(np.sum(past_ok))
    
    if denom == 0:
        return 0.5  # Return default value
    
    num = int(np.sum(B[past_ok, n] == b))
    p = num / denom
    return float(np.clip(p, 1e-12, 1.0 - 1e-12))

# ---------- Read data from bin file ----------
def read_bin_file(filename: str, max_bits: int = 200000) -> np.ndarray:
    """
    Read bin file, each byte stores one bit
    Return array in ±1 format
    """
    try:
        # Check if file exists
        if not os.path.exists(filename):
            print(f"  ERROR: File {filename} does not exist!", file=sys.stderr)
            return np.array([])
            
        with open(filename, 'rb') as f:
            data = f.read()
        
        if len(data) == 0:
            print(f"  ERROR: File {filename} is empty!", file=sys.stderr)
            return np.array([])
        
        # Limit data for faster computation
        if len(data) > max_bits:
            data = data[:max_bits]
        
        # Convert bytes to ±1
        bits = np.frombuffer(data, dtype=np.uint8)
        
        # Check data range
        unique_vals = np.unique(bits)
        
        # Convert based on actual values
        if len(unique_vals) == 2 and 0 in unique_vals and 1 in unique_vals:
            # Standard case: 0 and 1
            bits_converted = np.where(bits == 0, -1, 1)
        elif len(unique_vals) == 2 and 0 in unique_vals and 255 in unique_vals:
            # Possibly 0 and 255
            bits_converted = np.where(bits == 0, -1, 1)
        else:
            # Other cases, use LSB
            bits_converted = np.where(bits & 1 == 0, -1, 1)
        
        return bits_converted
    
    except Exception as e:
        print(f"  ERROR reading file {filename}: {e}", file=sys.stderr)
        return np.array([])

# ---------- Load parameters from CSV ----------
def load_parameters_from_csv(csv_file: str) -> Dict:
    """
    Load parameters from CSV file
    Expected format: Frequency,Folder,N,p_hat,r_u,r_f,beta,gamma,A,K_ru,R2
    """
    parameters = {}
    
    try:
        with open(csv_file, 'r', encoding='utf-8-sig') as f:
            reader = csv.DictReader(f)
            for row in reader:
                freq = row['Frequency']
                # Store all parameters for this frequency
                parameters[freq] = {
                    'Frequency': freq,
                    'Folder': row['Folder'],
                    'N': int(row['N']),
                    'p_hat': float(row['p_hat']),
                    'r_u': float(row['r_u']),
                    'r_f': float(row['r_f']),
                    'beta': float(row['beta']),
                    'gamma': float(row['gamma']),
                    'A': float(row['A']),
                    'K_ru': float(row['K_ru']),
                    'R2': float(row['R2'])
                }
        
        print(f"Loaded parameters for {len(parameters)} frequencies from {csv_file}")
        return parameters
    except Exception as e:
        print(f"Error loading parameters from CSV: {e}", file=sys.stderr)
        return {}

# ---------- Determine b based on r_u ----------
def determine_b_from_r_u(r_u: float) -> int:
    """
    Determine b value based on r_u:
    - If r_u is negative, b = -1
    - If r_u is positive, b = 1
    - If r_u is zero, b = -1 (default)
    """
    if r_u < 0:
        return -1
    elif r_u > 0:
        return 1
    else:
        return -1  # Default when r_u is exactly 0

# ---------- Analyze bin file with parameters ----------
def analyze_bin_file_with_params(filename: str, n_history: int, 
                                alpha: float, beta: float, 
                                r_u: float, mc_samples: int = 50000) -> Dict:
    """
    Analyze bin file with specific parameters
    b is determined automatically based on r_u
    r value is r_u from the CSV file
    """
    # Read data
    bits = read_bin_file(filename, max_bits=200000)
    if len(bits) == 0:
        return {
            'filename': filename,
            'hmin_linear': 0.0,
            'hmin_mc': 0.0,
            'p_linear': 0.5,
            'p_mc': 0.5,
            'error': 'Failed to read file'
        }
    
    # Use r_u as the r parameter
    r = r_u
    
    # Determine b based on r_u
    b = determine_b_from_r_u(r_u)
    
    # Method 1: First-order approximation
    p_linear = p_cond_first_order_all_same(n_history, r, alpha, beta, b)
    hmin_linear = hmin_from_p(p_linear)
    
    # Method 2: Monte Carlo
    p_mc = p_cond_monte_carlo_all_same(
        n_history, r, alpha, beta, b,
        num_samples=mc_samples, seed=42
    )
    hmin_mc = hmin_from_p(p_mc)
    
    # Calculate bias from data
    p1_actual = np.mean(bits == 1)
    bias = abs(p1_actual - 0.5)
    
    return {
        'filename': filename,
        'hmin_linear': hmin_linear,
        'hmin_mc': hmin_mc,
        'p_linear': p_linear,
        'p_mc': p_mc,
        'r': r,
        'r_u': r_u,
        'b': b,
        'b_determination': 'from_r_u',
        'p1_actual': p1_actual,
        'bias': bias,
        'alpha': alpha,
        'beta': beta,
        'n_history': n_history,
        'bits_analyzed': len(bits),
        'error': None
    }

# ---------- Save results to CSV ----------
def save_results_to_csv(results: List[Dict], filename: str, n_history: int = None, include_params: bool = False):
    """Save analysis results to CSV file"""
    if not results:
        print("No results to save", file=sys.stderr)
        return
    
    try:
        # Determine field names
        fieldnames = ['Frequency', '90B_Result', 'Linear_Estimate', 'MonteCarlo_Estimate']
        if include_params:
            fieldnames.extend(['p_hat', 'r_u', 'alpha', 'beta', 'b', 'n_history'])
        
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            for result in results:
                # Extract frequency from filename
                freq = result['filename'].split('_')[1].replace('G', 'GHz')
                
                row_data = {
                    'Frequency': freq,
                    '90B_Result': result['hmin_90b'],
                    'Linear_Estimate': result['hmin_linear'],
                    'MonteCarlo_Estimate': result['hmin_mc']
                }
                
                if include_params:
                    row_data.update({
                        'p_hat': result.get('p_hat', 0.5),
                        'r_u': result.get('r_u', 0.0),
                        'alpha': result.get('alpha', 0.0),
                        'beta': result.get('beta', 0.0),
                        'b': result.get('b', 0),
                        'n_history': n_history if n_history is not None else result.get('n_history', 8)
                    })
                
                writer.writerow(row_data)
        
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results to CSV: {e}", file=sys.stderr)

# ---------- Generate TeX table for single n_history ----------
def generate_tex_table(results: List[Dict], filename: str, n_history: int = None):
    """Generate LaTeX table from results for a single n_history"""
    try:
        with open(filename, 'w') as f:
            f.write("% LaTeX table generated by min-entropy analysis\n")
            f.write("% Date: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
            f.write("\\begin{table}[htbp]\n")
            f.write("\\centering\n")
            
            # Add n_history to caption if provided
            if n_history is not None:
                f.write(f"\\caption{{Comparison of Min-Entropy Estimation Methods (n={n_history})}}\n")
            else:
                f.write("\\caption{Comparison of Min-Entropy Estimation Methods}\n")
                
            f.write("\\label{tab:entropy_comparison}\n")
            f.write("\\begin{tabular}{cccc}\n")
            f.write("\\hline\n")
            f.write("Frequency & 90B Test & Linear Estimate & Monte Carlo Estimate \\\\\n")
            f.write("\\hline\n")
            
            for result in results:
                freq = result['filename'].split('_')[1].replace('G', 'GHz')
                f.write(f"{freq} & {result['hmin_90b']:.6f} & {result['hmin_linear']:.6f} & {result['hmin_mc']:.6f} \\\\\n")
            
            f.write("\\hline\n")
            f.write("\\end{tabular}\n")
            f.write("\\end{table}\n")
        
        print(f"TeX table saved to {filename}")
    except Exception as e:
        print(f"Error generating TeX table: {e}", file=sys.stderr)

# ---------- Generate TeX table with merged cells ----------
def generate_merged_format_tex_table(multi_results: Dict[int, List[Dict]], filename: str):
    """Generate LaTeX table with merged Linear Estimate and Monte Carlo Estimate headers"""
    try:
        n_vals = sorted(multi_results.keys())  # Should be [1, 3, 5]
        
        with open(filename, 'w') as f:
            f.write("% LaTeX table generated by min-entropy analysis\n")
            f.write("% Date: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
            f.write("\\begin{table}[htbp]\n")
            f.write("\\centering\n")
            f.write("\\caption{Min-Entropy Estimation Results for Different n Values}\n")
            f.write("\\label{tab:entropy_results_merged}\n")
            
            # Create table with correct column format
            # Format: l (Frequency), c (90B Test), then 3 c for Linear Estimate, then 3 c for Monte Carlo Estimate
            col_format = "l" + "c" * (1 + 3 + 3)  # l for Frequency, then 7 c columns
            
            f.write(f"\\begin{{tabular}}{{{col_format}}}\n")
            f.write("\\hline\n")
            
            # First header row with merged cells
            # Frequency and 90B Test span 2 rows, Linear Estimate spans 3 columns, Monte Carlo Estimate spans 3 columns
            f.write("\\multirow{2}{*}{Frequency} & \\multirow{2}{*}{90B Test} & \\multicolumn{3}{c}{Linear Estimate} & \\multicolumn{3}{c}{Monte Carlo Estimate} \\\\\n")
            
            # Second header row - n values under Linear and Monte Carlo
            f.write("\\cline{3-8}\n")
            f.write(" & & n=1 & n=3 & n=5 & n=1 & n=3 & n=5 \\\\\n")
            
            f.write("\\hline\n")
            
            # Get frequencies from the first n value's results
            first_n = n_vals[0]
            first_results = multi_results[first_n]
            
            # For each frequency, create a row
            for result in first_results:
                freq = result['filename'].split('_')[1].replace('G', 'GHz')
                row = f"{freq} & {result['hmin_90b']:.6f}"
                
                # Add linear estimates for all n values
                for n in n_vals:
                    n_results = multi_results[n]
                    n_result = next((r for r in n_results if r['filename'] == result['filename']), None)
                    
                    if n_result:
                        row += f" & {n_result['hmin_linear']:.6f}"
                    else:
                        row += " & -"
                
                # Add Monte Carlo estimates for all n values
                for n in n_vals:
                    n_results = multi_results[n]
                    n_result = next((r for r in n_results if r['filename'] == result['filename']), None)
                    
                    if n_result:
                        row += f" & {n_result['hmin_mc']:.6f}"
                    else:
                        row += " & -"
                
                row += " \\\\\n"
                f.write(row)
            
            f.write("\\hline\n")
            f.write("\\end{tabular}\n")
            f.write("\\end{table}\n")
        
        print(f"Merged format TeX table saved to {filename}")
    except Exception as e:
        print(f"Error generating merged format TeX table: {e}", file=sys.stderr)

# ---------- Generate Excel table with merged cells ----------
def generate_merged_format_excel_table(multi_results: Dict[int, List[Dict]], filename: str):
    """Generate Excel table with merged Linear Estimate and Monte Carlo Estimate headers"""
    try:
        # Try to import openpyxl for Excel generation
        try:
            from openpyxl import Workbook
            from openpyxl.styles import Alignment, Border, Side, Font, PatternFill
            from openpyxl.utils import get_column_letter
        except ImportError:
            print("openpyxl not installed. Cannot generate Excel file.")
            print("Please install openpyxl: pip install openpyxl")
            return
        
        n_vals = sorted(multi_results.keys())  # Should be [1, 3, 5]
        
        # Create a new workbook
        wb = Workbook()
        ws = wb.active
        ws.title = "Merged Format Results"
        
        # Define styles
        header_font = Font(bold=True, size=12)
        header_fill = PatternFill(start_color="C0C0C0", end_color="C0C0C0", fill_type="solid")
        subheader_font = Font(bold=True, size=11)
        subheader_fill = PatternFill(start_color="E0E0E0", end_color="E0E0E0", fill_type="solid")
        center_alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
        border = Border(
            left=Side(style="thin"),
            right=Side(style="thin"),
            top=Side(style="thin"),
            bottom=Side(style="thin")
        )
        
        # Write headers with merged cells
        # First row: Frequency (merged A1:A2), 90B Test (merged B1:B2), Linear Estimate (merged C1:E1), Monte Carlo Estimate (merged F1:H1)
        ws.merge_cells(start_row=1, start_column=1, end_row=2, end_column=1)  # Frequency
        ws.cell(row=1, column=1, value="Frequency").font = header_font
        ws.cell(row=1, column=1).fill = header_fill
        ws.cell(row=1, column=1).alignment = center_alignment
        ws.cell(row=1, column=1).border = border
        
        ws.merge_cells(start_row=1, start_column=2, end_row=2, end_column=2)  # 90B Test
        ws.cell(row=1, column=2, value="90B Test").font = header_font
        ws.cell(row=1, column=2).fill = header_fill
        ws.cell(row=1, column=2).alignment = center_alignment
        ws.cell(row=1, column=2).border = border
        
        # Linear Estimate header (spanning 3 columns in row 1)
        ws.merge_cells(start_row=1, start_column=3, end_row=1, end_column=5)
        ws.cell(row=1, column=3, value="Linear Estimate").font = header_font
        ws.cell(row=1, column=3).fill = header_fill
        ws.cell(row=1, column=3).alignment = center_alignment
        ws.cell(row=1, column=3).border = border
        
        # Monte Carlo Estimate header (spanning 3 columns in row 1)
        ws.merge_cells(start_row=1, start_column=6, end_row=1, end_column=8)
        ws.cell(row=1, column=6, value="Monte Carlo Estimate").font = header_font
        ws.cell(row=1, column=6).fill = header_fill
        ws.cell(row=1, column=6).alignment = center_alignment
        ws.cell(row=1, column=6).border = border
        
        # Second row: n values under Linear Estimate and Monte Carlo Estimate
        # Linear Estimate n values (row 2, columns 3-5)
        for i, n in enumerate(n_vals):
            col = 3 + i
            ws.cell(row=2, column=col, value=f"n={n}").font = subheader_font
            ws.cell(row=2, column=col).fill = subheader_fill
            ws.cell(row=2, column=col).alignment = center_alignment
            ws.cell(row=2, column=col).border = border
        
        # Monte Carlo Estimate n values (row 2, columns 6-8)
        for i, n in enumerate(n_vals):
            col = 6 + i
            ws.cell(row=2, column=col, value=f"n={n}").font = subheader_font
            ws.cell(row=2, column=col).fill = subheader_fill
            ws.cell(row=2, column=col).alignment = center_alignment
            ws.cell(row=2, column=col).border = border
        
        # Get frequencies from the first n value's results
        first_n = n_vals[0]
        first_results = multi_results[first_n]
        
        # Write data rows starting from row 3
        for row_idx, result in enumerate(first_results, start=3):
            freq = result['filename'].split('_')[1].replace('G', 'GHz')
            
            # Frequency
            ws.cell(row=row_idx, column=1, value=freq).alignment = center_alignment
            ws.cell(row=row_idx, column=1).border = border
            
            # 90B Test result
            ws.cell(row=row_idx, column=2, value=result['hmin_90b']).alignment = center_alignment
            ws.cell(row=row_idx, column=2).border = border
            
            # Linear estimates for all n values
            for i, n in enumerate(n_vals):
                col = 3 + i
                n_results = multi_results[n]
                n_result = next((r for r in n_results if r['filename'] == result['filename']), None)
                
                if n_result:
                    ws.cell(row=row_idx, column=col, value=n_result['hmin_linear']).alignment = center_alignment
                else:
                    ws.cell(row=row_idx, column=col, value="N/A").alignment = center_alignment
                
                ws.cell(row=row_idx, column=col).border = border
            
            # Monte Carlo estimates for all n values
            for i, n in enumerate(n_vals):
                col = 6 + i
                n_results = multi_results[n]
                n_result = next((r for r in n_results if r['filename'] == result['filename']), None)
                
                if n_result:
                    ws.cell(row=row_idx, column=col, value=n_result['hmin_mc']).alignment = center_alignment
                else:
                    ws.cell(row=row_idx, column=col, value="N/A").alignment = center_alignment
                
                ws.cell(row=row_idx, column=col).border = border
        
        # Adjust column widths
        column_widths = {
            1: 12,  # Frequency
            2: 12,  # 90B Test
            3: 12,  # Linear n=1
            4: 12,  # Linear n=3
            5: 12,  # Linear n=5
            6: 12,  # Monte Carlo n=1
            7: 12,  # Monte Carlo n=3
            8: 12   # Monte Carlo n=5
        }
        
        for col, width in column_widths.items():
            col_letter = get_column_letter(col)
            ws.column_dimensions[col_letter].width = width
        
        # Save the workbook
        wb.save(filename)
        print(f"Merged format Excel table saved to {filename}")
        
    except ImportError:
        print("openpyxl not available. Skipping Excel generation.")
    except Exception as e:
        print(f"Error generating merged format Excel table: {e}", file=sys.stderr)

# ---------- Generate CSV table with merged format ----------
def generate_merged_format_csv_table(multi_results: Dict[int, List[Dict]], filename: str):
    """Generate CSV table with merged format header"""
    try:
        n_vals = sorted(multi_results.keys())  # Should be [1, 3, 5]
        
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            
            # Write header rows with merged format
            # First header row
            header1 = ['Frequency', '90B Test', 'Linear Estimate', '', '', 'Monte Carlo Estimate', '', '']
            writer.writerow(header1)
            
            # Second header row
            header2 = ['', '', 'n=1', 'n=3', 'n=5', 'n=1', 'n=3', 'n=5']
            writer.writerow(header2)
            
            # Extract frequencies from first result list
            first_n = list(multi_results.keys())[0]
            first_results = multi_results[first_n]
            
            # For each frequency, create a row
            for result in first_results:
                freq = result['filename'].split('_')[1].replace('G', 'GHz')
                row = [freq, result['hmin_90b']]
                
                # Add linear estimates for all n values
                for n in n_vals:
                    n_results = multi_results[n]
                    n_result = next((r for r in n_results if r['filename'] == result['filename']), None)
                    
                    if n_result:
                        row.append(n_result['hmin_linear'])
                    else:
                        row.append('N/A')
                
                # Add Monte Carlo estimates for all n values
                for n in n_vals:
                    n_results = multi_results[n]
                    n_result = next((r for r in n_results if r['filename'] == result['filename']), None)
                    
                    if n_result:
                        row.append(n_result['hmin_mc'])
                    else:
                        row.append('N/A')
                
                writer.writerow(row)
        
        print(f"Merged format CSV table saved to {filename}")
    except Exception as e:
        print(f"Error generating merged format CSV table: {e}", file=sys.stderr)

# ---------- Visualization functions ----------
def create_visualizations(results: List[Dict]):
    """Create visualizations for the analysis results"""
    try:
        import matplotlib.pyplot as plt
        import matplotlib
        matplotlib.rcParams.update({'font.size': 10})
        
        # Extract data for plotting
        frequencies = []
        hmin_90b = []
        hmin_linear = []
        hmin_mc = []
        alphas = []
        betas = []
        r_vals = []
        
        for result in results:
            if result['error'] is None:
                freq = result['filename'].split('_')[1].replace('G', 'GHz')
                frequencies.append(freq)
                hmin_90b.append(result['hmin_90b'])
                hmin_linear.append(result['hmin_linear'])
                hmin_mc.append(result['hmin_mc'])
                alphas.append(result.get('alpha', 0.0))
                betas.append(result.get('beta', 0.0))
                r_vals.append(result.get('r', 0.0))
        
        if not frequencies:
            print("No valid results for visualization")
            return
        
        # Create figure with subplots
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle('Min-Entropy Analysis Results', fontsize=14, fontweight='bold')
        
        # 1. Main comparison line plot
        ax1 = axes[0, 0]
        ax1.plot(frequencies, hmin_90b, 'o-', linewidth=2, markersize=8, 
                label='90B Test', color='blue')
        ax1.plot(frequencies, hmin_linear, 's-', linewidth=2, markersize=8,
                label='Linear Estimate', color='green')
        ax1.plot(frequencies, hmin_mc, '^-', linewidth=2, markersize=8,
                label='Monte Carlo', color='red')
        
        ax1.set_xlabel('Frequency')
        ax1.set_ylabel('Min-Entropy (bits)')
        ax1.set_title('Min-Entropy Comparison')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 2. Difference from 90B results
        ax2 = axes[0, 1]
        diff_linear = np.abs(np.array(hmin_linear) - np.array(hmin_90b))
        diff_mc = np.abs(np.array(hmin_mc) - np.array(hmin_90b))
        
        x = np.arange(len(frequencies))
        width = 0.35
        
        bars1 = ax2.bar(x - width/2, diff_linear, width, label='Linear Difference', 
                       color='green', alpha=0.7)
        bars2 = ax2.bar(x + width/2, diff_mc, width, label='Monte Carlo Difference',
                       color='red', alpha=0.7)
        
        ax2.set_xlabel('Frequency')
        ax2.set_ylabel('Absolute Difference from 90B (bits)')
        ax2.set_title('Estimation Errors')
        ax2.set_xticks(x)
        ax2.set_xticklabels(frequencies)
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Add value labels on bars
        for bars in [bars1, bars2]:
            for bar in bars:
                height = bar.get_height()
                ax2.text(bar.get_x() + bar.get_width()/2., height,
                        f'{height:.3f}', ha='center', va='bottom', fontsize=8)
        
        # 3. Parameter values
        ax3 = axes[0, 2]
        
        x = np.arange(len(frequencies))
        width = 0.25
        
        bars1 = ax3.bar(x - width, alphas, width, label='Alpha (r_f)', 
                       color='blue', alpha=0.7)
        bars2 = ax3.bar(x, betas, width, label='Beta', 
                       color='orange', alpha=0.7)
        bars3 = ax3.bar(x + width, np.abs(r_vals), width, label='|r_u|', 
                       color='purple', alpha=0.7)
        
        ax3.set_xlabel('Frequency')
        ax3.set_ylabel('Parameter Value')
        ax3.set_title('Model Parameters by Frequency')
        ax3.set_xticks(x)
        ax3.set_xticklabels(frequencies)
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        # 4. Scatter plot: Linear vs Monte Carlo estimates
        ax4 = axes[1, 0]
        
        # Perfect correlation line
        min_val = min(min(hmin_linear), min(hmin_mc))
        max_val = max(max(hmin_linear), max(hmin_mc))
        ax4.plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.5, label='y=x')
        
        # Scatter points with frequency labels
        scatter = ax4.scatter(hmin_linear, hmin_mc, c=range(len(frequencies)), 
                             cmap='viridis', s=100, alpha=0.7)
        
        # Add frequency labels to points
        for i, freq in enumerate(frequencies):
            ax4.annotate(freq, (hmin_linear[i], hmin_mc[i]), 
                        xytext=(5, 5), textcoords='offset points', fontsize=8)
        
        ax4.set_xlabel('Linear Estimate (bits)')
        ax4.set_ylabel('Monte Carlo Estimate (bits)')
        ax4.set_title('Linear vs Monte Carlo Estimates')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        
        # 5. Relative error comparison
        ax5 = axes[1, 1]
        
        # Calculate relative errors
        rel_error_linear = diff_linear / np.array(hmin_90b) * 100
        rel_error_mc = diff_mc / np.array(hmin_90b) * 100
        
        x = np.arange(len(frequencies))
        bar_width = 0.35
        
        bars_linear = ax5.bar(x - bar_width/2, rel_error_linear, bar_width, 
                             label='Linear Error (%)', color='green', alpha=0.7)
        bars_mc = ax5.bar(x + bar_width/2, rel_error_mc, bar_width, 
                         label='Monte Carlo Error (%)', color='red', alpha=0.7)
        
        ax5.set_xlabel('Frequency')
        ax5.set_ylabel('Relative Error (%)')
        ax5.set_title('Relative Errors Compared to 90B')
        ax5.set_xticks(x)
        ax5.set_xticklabels(frequencies)
        ax5.legend()
        ax5.grid(True, alpha=0.3)
        
        # Add value labels on bars
        for bars in [bars_linear, bars_mc]:
            for bar in bars:
                height = bar.get_height()
                ax5.text(bar.get_x() + bar.get_width()/2., height,
                        f'{height:.1f}%', ha='center', va='bottom', fontsize=8)
        
        # 6. Correlation heatmap
        ax6 = axes[1, 2]
        
        # Create correlation matrix
        data_matrix = np.array([hmin_90b, hmin_linear, hmin_mc, alphas, betas, np.abs(r_vals)])
        corr_matrix = np.corrcoef(data_matrix)
        
        # Labels for the matrix
        labels = ['90B', 'Linear', 'MC', 'Alpha', 'Beta', '|r_u|']
        
        # Create heatmap
        im = ax6.imshow(corr_matrix, cmap='RdYlBu', vmin=-1, vmax=1)
        
        # Add text annotations
        for i in range(len(labels)):
            for j in range(len(labels)):
                text = ax6.text(j, i, f'{corr_matrix[i, j]:.2f}',
                              ha='center', va='center', color='black', fontsize=8)
        
        ax6.set_xticks(range(len(labels)))
        ax6.set_yticks(range(len(labels)))
        ax6.set_xticklabels(labels, rotation=45)
        ax6.set_yticklabels(labels)
        ax6.set_title('Correlation Matrix')
        
        # Add colorbar
        plt.colorbar(im, ax=ax6, fraction=0.046, pad=0.04)
        
        plt.tight_layout()
        
        # Save the figure
        plt.savefig('entropy_analysis_visualizations.png', dpi=300, bbox_inches='tight')
        print("Visualizations saved as 'entropy_analysis_visualizations.png'")
        
        plt.show()
        
    except ImportError:
        print("Matplotlib not available. Skipping visualizations.")
    except Exception as e:
        print(f"Error creating visualizations: {e}")

# ---------- Main program ----------
def main():
    """Main function: analyze all files and generate comparison tables and visualizations"""
    # File list and corresponding 90B test results
    file_data = [
        ("1.3V_1G.bin", 0.613673),
        ("1.3V_2G.bin", 0.567689),
        ("1.3V_3G.bin", 0.519764),
        ("1.3V_4G.bin", 0.513656),
        ("1.3V_5G.bin", 0.328293)
    ]
    
    # Load parameters from CSV file
    parameters = load_parameters_from_csv("parameter_table.csv")
    
    if not parameters:
        print("ERROR: Could not load parameters from CSV file", file=sys.stderr)
        return
    
    # Analysis history length - only analyze n=1, 3, 5
    n_history_list = [1, 3, 5]
    
    # Monte Carlo samples
    mc_samples = 10000000
    
    print("=" * 80)
    print("MIN-ENTROPY ANALYSIS WITH DYNAMIC PARAMETERS")
    print("=" * 80)
    print(f"Analysis started at: {datetime.datetime.now()}")
    print(f"Analyzing n_history values: {n_history_list}")
    print(f"b is automatically determined from r_u sign")
    print("=" * 80)
    
    # Store results for each n_history
    multi_results = {}
    
    # Analyze for each n_history value
    for n_history in n_history_list:
        print(f"\n{'='*40}")
        print(f"ANALYZING WITH n = {n_history}")
        print(f"{'='*40}")
        
        # Store results for this n_history
        all_results = []
        
        # Analyze each file with its corresponding parameters
        for filename, hmin_90b in file_data:
            # Extract frequency from filename to match with parameter table
            freq_from_file = filename.split('_')[1].replace('G', 'GHz')
            
            print(f"\nAnalyzing file: {filename}")
            print(f"90B test result: {hmin_90b:.6f}")
            print(f"Extracted frequency: {freq_from_file}")
            
            # Get parameters for this frequency
            if freq_from_file in parameters:
                params = parameters[freq_from_file]
                print(f"Using parameters for {freq_from_file}:")
                print(f"  r_u (used as r): {params['r_u']:.6f}")
                print(f"  r_f (alpha): {params['r_f']:.6f}")
                print(f"  beta: {params['beta']:.6f}")
                print(f"  p_hat: {params['p_hat']:.6f} (for reference only)")
                
                # Determine b based on r_u
                b = determine_b_from_r_u(params['r_u'])
                print(f"  Determined b: {b} (from r_u sign)")
                
                # Analyze with parameters - USING r_u AS r
                result = analyze_bin_file_with_params(
                    filename=filename,
                    n_history=n_history,
                    alpha=params['r_f'],  # Use r_f as alpha
                    beta=params['beta'],   # Use beta from CSV
                    r_u=params['r_u'],     # Use r_u as the r parameter
                    mc_samples=mc_samples
                )
                
                # Add 90B result and p_hat for reference
                result['hmin_90b'] = hmin_90b
                result['p_hat'] = params['p_hat']  # Store for reference
                result['param_source'] = freq_from_file
                
                all_results.append(result)
                
                print(f"  Linear estimate: p={result['p_linear']:.6f}, H_min={result['hmin_linear']:.6f}")
                print(f"  Monte Carlo: p={result['p_mc']:.6f}, H_min={result['hmin_mc']:.6f}")
            else:
                print(f"WARNING: No parameters found for frequency {freq_from_file}")
                continue
        
        # Store results for this n_history
        multi_results[n_history] = all_results
        
        # Print summary table for this n_history
        print(f"\n{'='*40}")
        print(f"RESULTS SUMMARY FOR n = {n_history}")
        print(f"{'='*40}")
        print(f"{'Frequency':<12} {'90B Test':<12} {'Linear':<12} {'Monte Carlo':<12} {'Diff (Lin)':<12} {'Diff (MC)':<12}")
        print("-" * 80)
        
        total_diff_linear = 0
        total_diff_mc = 0
        valid_results = 0
        
        for result in all_results:
            if result['error'] is None:
                freq = result['filename'].split('_')[1].replace('G', 'GHz')
                diff_linear = abs(result['hmin_linear'] - result['hmin_90b'])
                diff_mc = abs(result['hmin_mc'] - result['hmin_90b'])
                
                total_diff_linear += diff_linear
                total_diff_mc += diff_mc
                valid_results += 1
                
                print(f"{freq:<12} {result['hmin_90b']:<12.6f} {result['hmin_linear']:<12.6f} {result['hmin_mc']:<12.6f} {diff_linear:<12.6f} {diff_mc:<12.6f}")
        
        if valid_results > 0:
            avg_diff_linear = total_diff_linear / valid_results
            avg_diff_mc = total_diff_mc / valid_results
            
            print("-" * 80)
            print(f"{'Average Diff':<12} {'':<12} {'':<12} {'':<12} {avg_diff_linear:<12.6f} {avg_diff_mc:<12.6f}")
        
        # Save results to CSV files for this n_history
        save_results_to_csv(all_results, f"entropy_results_n{n_history}_summary.csv", n_history=n_history, include_params=False)
        save_results_to_csv(all_results, f"entropy_results_n{n_history}_detailed.csv", n_history=n_history, include_params=True)
        
        # Generate TeX tables for this n_history
        generate_tex_table(all_results, f"entropy_results_table_n{n_history}.tex", n_history=n_history)
    
    # Generate tables with merged format
    print(f"\n{'='*80}")
    print("GENERATING TABLES WITH MERGED HEADER FORMAT")
    print(f"{'='*80}")
    
    # Generate CSV table with merged format
    generate_merged_format_csv_table(multi_results, "entropy_results_merged_format.csv")
    
    # Generate Excel table with merged format
    generate_merged_format_excel_table(multi_results, "entropy_results_merged_format.xlsx")
    
    # Generate TeX table with merged format
    generate_merged_format_tex_table(multi_results, "entropy_results_merged_format.tex")
    
    # Create visualizations for the first n_history (n=1)
    print(f"\n{'='*80}")
    print("CREATING VISUALIZATIONS FOR n=1")
    print(f"{'='*80}")
    
    if 1 in multi_results:
        create_visualizations(multi_results[1])
    
    print(f"\n{'='*80}")
    print("ANALYSIS COMPLETE")
    print(f"{'='*80}")
    print("Generated files:")
    
    # List files for each n_history
    for n_history in n_history_list:
        print(f"  For n={n_history}:")
        print(f"    1. entropy_results_n{n_history}_summary.csv")
        print(f"    2. entropy_results_n{n_history}_detailed.csv")
        print(f"    3. entropy_results_table_n{n_history}.tex")
    
    print(f"\n  Multi-n tables with merged header format:")
    print(f"    1. entropy_results_merged_format.csv")
    print(f"    2. entropy_results_merged_format.xlsx (Excel format with merged cells)")
    print(f"    3. entropy_results_merged_format.tex (TeX format with merged cells)")
    print(f"\n  Visualizations (for n=1):")
    print(f"    1. entropy_analysis_visualizations.png (if matplotlib available)")
    print(f"{'='*80}")

if __name__ == "__main__":
    main()

Loaded parameters for 5 frequencies from parameter_table.csv
MIN-ENTROPY ANALYSIS WITH DYNAMIC PARAMETERS
Analysis started at: 2026-01-15 13:51:53.491581
Analyzing n_history values: [1, 3, 5]
b is automatically determined from r_u sign

ANALYZING WITH n = 1

Analyzing file: 1.3V_1G.bin
90B test result: 0.613673
Extracted frequency: 1GHz.bin

Analyzing file: 1.3V_2G.bin
90B test result: 0.567689
Extracted frequency: 2GHz.bin

Analyzing file: 1.3V_3G.bin
90B test result: 0.519764
Extracted frequency: 3GHz.bin

Analyzing file: 1.3V_4G.bin
90B test result: 0.513656
Extracted frequency: 4GHz.bin

Analyzing file: 1.3V_5G.bin
90B test result: 0.328293
Extracted frequency: 5GHz.bin

RESULTS SUMMARY FOR n = 1
Frequency    90B Test     Linear       Monte Carlo  Diff (Lin)   Diff (MC)   
--------------------------------------------------------------------------------
TeX table saved to entropy_results_table_n1.tex

ANALYZING WITH n = 3

Analyzing file: 1.3V_1G.bin
90B test result: 0.613673
Extrac

No results to save
No results to save
No results to save
No results to save
No results to save
No results to save


No valid results for visualization

ANALYSIS COMPLETE
Generated files:
  For n=1:
    1. entropy_results_n1_summary.csv
    2. entropy_results_n1_detailed.csv
    3. entropy_results_table_n1.tex
  For n=3:
    1. entropy_results_n3_summary.csv
    2. entropy_results_n3_detailed.csv
    3. entropy_results_table_n3.tex
  For n=5:
    1. entropy_results_n5_summary.csv
    2. entropy_results_n5_detailed.csv
    3. entropy_results_table_n5.tex

  Multi-n tables with merged header format:
    1. entropy_results_merged_format.csv
    2. entropy_results_merged_format.xlsx (Excel format with merged cells)
    3. entropy_results_merged_format.tex (TeX format with merged cells)

  Visualizations (for n=1):
    1. entropy_analysis_visualizations.png (if matplotlib available)
